In [6]:
from src.experimental_config import *
from src.policies import *
from src.simulator import *
from src.forecasting import *

import os

# delete all files in the experiments folder
for filename in os.listdir('./experiments'):
    file_path = os.path.join('./experiments', filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))


generate_experiments(
    price_data_dir = './raw_data/spx_stock_prices.parquet',
    covariates = None,
    num_experiments = 50,
    output_dir = './experiments',
    min_horizon = 30,
    max_horizon = 90,
    max_num_stocks = 30,
    min_num_stocks = 5,
    min_tx_cost = 2,
    max_tx_cost = 30,
    max_budget = 350000,
    min_budget = 5000,
    forecast_model='NBEATS'
)

100%|██████████| 50/50 [00:11<00:00,  4.23it/s]


In [7]:
import pickle
from time import time
# from tqdm.notebook import tqdm
results = []
# get the first file in the experiments folder
# pbar = tqdm(os.listdir('./experiments'))
for filename in os.listdir('./experiments'):
    try:
        file_path = os.path.join('./experiments', filename)
        with open(file_path, "rb") as f:
            exp = pickle.load(f)
        # display the experiment id in the progress bar
        print(f"Running {exp.exp_id}")
        t1 = time()
        policy = DirectionalTradingPolicy(exp, verbose=False)
        simulator = MarketSimulator(exp, policy, verbose=False)
        result = simulator.run()
        t2 = time()
        results.append({
            "runtime": t2 - t1,
            "total_trades": simulator.total_trades,
            "feasibility": simulator.status,
            "horizon": exp.horizon
        })
        del exp
        del policy
        del simulator
        gc.collect()
    except Exception as e:
        print(f"\tFailed to run {exp.exp_id}. Reason: {e}")
        continue

# turn results into a dataframe
results_df = pd.DataFrame(results)

# plot runtime vs horizon
results_df.plot.scatter(x='horizon', y='runtime')


Global seed set to 1


Running 20180129_44_8_23_304181


Global seed set to 1


Running 20180209_33_8_11_156740


Global seed set to 1


...

Global seed set to 1


Running 20190103_45_8_11_339603


Global seed set to 1


Running 20190117_39_10_27_122789


KeyboardInterrupt: 

In [ ]:
exp.target_portfolio <= result

CB_79057      True
CTAS_23660    True
ESS_80681     True
HBAN_42906    True
NVR_79785     True
SBAC_86996    True
SBUX_77702    True
VTRS_20057    True
WRB_17743     True
cash          True
dtype: bool

In [ ]:
simulator.status

'Complete'